<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Корреляционный-анализ" data-toc-modified-id="Корреляционный-анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Корреляционный анализ</a></span><ul class="toc-item"><li><span><a href="#Столбцы-с-0-по-199" data-toc-modified-id="Столбцы-с-0-по-199-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Столбцы с 0 по 199</a></span></li><li><span><a href="#Столбцы-с-200-по-399" data-toc-modified-id="Столбцы-с-200-по-399-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Столбцы с 200 по 399</a></span></li><li><span><a href="#Столбцы-с-400-по-599" data-toc-modified-id="Столбцы-с-400-по-599-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Столбцы с 400 по 599</a></span></li><li><span><a href="#Столбцы-с-600-по-799" data-toc-modified-id="Столбцы-с-600-по-799-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Столбцы с 600 по 799</a></span></li><li><span><a href="#Столбцы-с-800-по-930" data-toc-modified-id="Столбцы-с-800-по-930-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Столбцы с 800 по 930</a></span></li></ul></li></ul></div>

# Отбор значимых непрерывных признаков

## Обзор данных

In [1]:
# импорт основных библиотек
import pandas as pd
import numpy as np

# импорт библиотек текущего модуля
from scipy.stats import spearmanr
import phik

# настройки
pd.set_option('display.max_columns', 500)

# пути
dataset_train_path = '../../data/dataset_train.parquet'
features_types_path = '../../data/features_types.json'
sample_submission_path = '../../data/sample_submission.csv'

# константы
RANDOM_STATE = 42


In [2]:
# читаем файлы
features_types = pd.read_json(features_types_path, orient='index')
sample_submission = pd.read_csv(sample_submission_path)


In [3]:
features_types.head()

,0
markers_0_1_cnt,numeric
markers_1_1_cnt,numeric
markers_2_1_cnt,numeric
markers_3_1_cnt,numeric
markers_4_1_cnt,numeric


In [4]:
# приводим features_types к читаемому виду
features_types = features_types \
    .reset_index() \
    .rename(columns={'index': 'feature', 0: 'type'})

# выводим на экран
features_types


,feature,type
0,markers_0_1_cnt,numeric
1,markers_1_1_cnt,numeric
2,markers_2_1_cnt,numeric
3,markers_3_1_cnt,numeric
4,markers_4_1_cnt,numeric
...,...,...
2771,markers_941_1_cnt,numeric
2772,markers_942_1_cnt,numeric
2773,markers_943_1_cnt,numeric
2774,markers_944_1_cnt,numeric


In [5]:
# распределение типов данных
features_types.type.value_counts()


type
numeric            2607
categorical_int     138
categorical_str      31
Name: count, dtype: int64

## Корреляционный анализ

### Столбцы с 0 по 199

In [6]:
def read_data(path: str, left: int, right: int):
    """
    Функция для считывания определенного количества столбцов из parquet файла.
    ---
    Принимает на вход str путь к файлу, int левую и int правую границу.
    Столбец с таргетом добавляется в конец автоматически!
    Функция возвращает pd.DataFrame.
    """
    # следующие т столбцов, которые будем считывать + target
    columns_to_read = features_types['feature'][left:right].to_list() + ['target']

    # считываем столбцы в файл
    data = pd.read_parquet(path, engine='pyarrow', columns=columns_to_read)

    return data


def variance_filter(data: pd.DataFrame, threshold: float):
    """
    Функция для проверки датасета на наличие константных (или около того) значений.
    ---
    Принимает на вход pd.DataFrame и float с значением порога для дисперсии.
    Функция возвращает отфильтрованный pd.DataFrame.
    """
    # порог для дисперсии
    threshold = threshold

    # считаем дисперсию для каждого признака
    variance = data.iloc[:, :-1].var()

    # отбираем те фичи, у которых var > threshold
    selected_features = variance[variance > threshold].index.tolist()

    # собираем в датасет
    filtered_data = data[selected_features + ['target']]
    
    return filtered_data
    

def check_corr(data: pd.DataFrame):
    """
    Функция для расчета корреляции Пирсона всех признаков с целевой переменной.
    ---
    Принимает на вход pd.DataFrame.
    Возвращает pd.Series с значением коэффициента корреляции для каждого признака.
    """
    # чекаем корреляцию с таргетом
    correlation = data.iloc[:, :-1].corrwith(data['target'])
    
    # сортируем
    correlation = correlation.sort_values(ascending=False)
    
    return correlation

In [7]:
# считываем необходимое количество столбцов
dataset_0_200 = read_data(dataset_train_path, 0, 200)
dataset_0_200.head()

,markers_0_1_cnt,markers_1_1_cnt,markers_2_1_cnt,markers_3_1_cnt,markers_4_1_cnt,markers_5_1_cnt,markers_6_1_cnt,markers_7_1_cnt,markers_8_1_cnt,markers_9_1_cnt,markers_10_1_cnt,markers_11_1_cnt,markers_12_1_cnt,markers_13_1_cnt,markers_14_1_cnt,markers_15_1_cnt,markers_16_1_cnt,markers_17_1_cnt,markers_18_1_cnt,markers_19_1_cnt,markers_20_1_cnt,markers_21_1_cnt,markers_22_1_cnt,markers_23_1_cnt,markers_24_1_cnt,markers_25_1_cnt,markers_26_1_cnt,markers_27_1_cnt,markers_28_1_cnt,markers_29_1_cnt,markers_30_1_cnt,markers_31_1_cnt,markers_32_1_cnt,markers_33_1_cnt,markers_34_1_cnt,markers_35_1_cnt,markers_36_1_cnt,markers_37_1_cnt,markers_38_1_cnt,markers_39_1_cnt,markers_40_1_cnt,markers_41_1_cnt,markers_42_1_cnt,markers_43_1_cnt,markers_44_1_cnt,markers_45_1_cnt,markers_46_1_cnt,markers_47_1_cnt,markers_48_1_cnt,markers_49_1_cnt,markers_50_1_cnt,markers_51_1_cnt,markers_52_1_cnt,markers_53_1_cnt,markers_54_1_cnt,markers_55_1_cnt,markers_56_1_cnt,markers_57_1_cnt,markers_58_1_cnt,markers_59_1_cnt,markers_60_1_cnt,markers_61_1_cnt,markers_62_1_cnt,markers_63_1_cnt,markers_64_1_cnt,markers_65_1_cnt,markers_66_1_cnt,markers_67_1_cnt,markers_68_1_cnt,markers_69_1_cnt,markers_70_1_cnt,markers_71_1_cnt,markers_72_1_cnt,markers_73_1_cnt,markers_74_1_cnt,markers_75_1_cnt,markers_76_1_cnt,markers_77_1_cnt,markers_78_1_cnt,markers_79_1_cnt,markers_80_1_cnt,markers_81_1_cnt,markers_82_1_cnt,markers_83_1_cnt,markers_84_1_cnt,markers_85_1_cnt,markers_86_1_cnt,markers_87_1_cnt,markers_88_1_cnt,markers_89_1_cnt,markers_90_1_cnt,markers_91_1_cnt,markers_92_1_cnt,markers_93_1_cnt,markers_94_1_cnt,markers_95_1_cnt,markers_96_1_cnt,markers_97_1_cnt,markers_98_1_cnt,markers_99_1_cnt,markers_100_1_cnt,markers_101_1_cnt,markers_102_1_cnt,markers_103_1_cnt,markers_104_1_cnt,markers_105_1_cnt,markers_106_1_cnt,markers_107_1_cnt,markers_108_1_cnt,markers_109_1_cnt,markers_110_1_cnt,markers_111_1_cnt,markers_112_1_cnt,markers_113_1_cnt,markers_114_1_cnt,markers_115_1_cnt,markers_116_1_cnt,markers_117_1_cnt,markers_118_1_cnt,markers_119_1_cnt,markers_120_1_cnt,markers_121_1_cnt,markers_122_1_cnt,markers_123_1_cnt,markers_124_1_cnt,markers_125_1_cnt,markers_126_1_cnt,markers_128_1_cnt,markers_129_1_cnt,markers_131_1_cnt,markers_132_1_cnt,markers_133_1_cnt,markers_135_1_cnt,markers_136_1_cnt,markers_137_1_cnt,markers_138_1_cnt,markers_139_1_cnt,markers_140_1_cnt,markers_141_1_cnt,markers_142_1_cnt,markers_143_1_cnt,markers_145_1_cnt,markers_146_1_cnt,markers_147_1_cnt,markers_148_1_cnt,markers_149_1_cnt,markers_150_1_cnt,markers_151_1_cnt,markers_154_1_cnt,markers_155_1_cnt,markers_156_1_cnt,markers_157_1_cnt,markers_158_1_cnt,markers_159_1_cnt,markers_160_1_cnt,markers_161_1_cnt,markers_162_1_cnt,markers_163_1_cnt,markers_164_1_cnt,markers_165_1_cnt,markers_166_1_cnt,markers_167_1_cnt,markers_168_1_cnt,markers_169_1_cnt,markers_170_1_cnt,markers_171_1_cnt,markers_172_1_cnt,markers_173_1_cnt,markers_174_1_cnt,markers_175_1_cnt,markers_176_1_cnt,markers_177_1_cnt,markers_178_1_cnt,markers_179_1_cnt,markers_180_1_cnt,markers_181_1_cnt,markers_182_1_cnt,markers_183_1_cnt,markers_184_1_cnt,markers_185_1_cnt,markers_186_1_cnt,markers_187_1_cnt,markers_188_1_cnt,markers_189_1_cnt,markers_190_1_cnt,markers_191_1_cnt,markers_192_1_cnt,markers_193_1_cnt,markers_194_1_cnt,markers_195_1_cnt,markers_196_1_cnt,markers_198_1_cnt,markers_199_1_cnt,markers_201_1_cnt,markers_202_1_cnt,markers_203_1_cnt,markers_204_1_cnt,markers_205_1_cnt,markers_206_1_cnt,markers_207_1_cnt,target
0,-0.149534,-0.096585,-0.400870,-0.362458,0.302917,-0.324892,-0.046691,-0.554345,-0.016534,-0.05408,-0.044653,-0.208968,-0.017454,-0.008052,-0.028391,-0.031395,-0.199957,-0.064366,-0.026839,-0.027484,-0.03147,-0.080534,-0.110653,-0.11212,-0.02527,-0.115151,-0.04685,-0.033911,-0.00794,-0.025816,-0.088375,-0.009532,-0.237766,-0.041626,-0.012926,-0.028445,-0.009258,-0.026347,-0.038496,-0.089040,-0.401337,-0.03564,-0.100471,-0.035238,-0.009976,-0.031354,-0.006144,-0.070189,-0.047567,-0.050881,-0.19008

In [8]:
# отбрасываем константные значения
dataset_0_200 = variance_filter(dataset_0_200, 0.5)


In [9]:
# считаем корреляцию с таргетом
corr_0_200 = check_corr(dataset_0_200)
corr_0_200

markers_40_1_cnt     0.014241
markers_32_1_cnt     0.010720
markers_103_1_cnt    0.009781
markers_2_1_cnt      0.009140
markers_171_1_cnt    0.008179
                       ...   
markers_65_1_cnt    -0.005823
markers_199_1_cnt   -0.006428
markers_72_1_cnt    -0.008867
markers_185_1_cnt   -0.009140
markers_122_1_cnt   -0.011679
Length: 197, dtype: float64

Очевидно значимых признаков нет

### Столбцы с 200 по 399

In [10]:
# считываем необходимое количество столбцов
dataset_200_400 = read_data(dataset_train_path, 200, 400)
dataset_200_400.head()


,markers_208_1_cnt,markers_209_1_cnt,markers_210_1_cnt,markers_211_1_cnt,markers_212_1_cnt,markers_213_1_cnt,markers_215_1_cnt,markers_216_1_cnt,markers_217_1_cnt,markers_218_1_cnt,markers_219_1_cnt,markers_220_1_cnt,markers_221_1_cnt,markers_222_1_cnt,markers_223_1_cnt,markers_224_1_cnt,markers_225_1_cnt,markers_226_1_cnt,markers_227_1_cnt,markers_228_1_cnt,markers_229_1_cnt,markers_230_1_cnt,markers_231_1_cnt,markers_232_1_cnt,markers_233_1_cnt,markers_234_1_cnt,markers_235_1_cnt,markers_236_1_cnt,markers_237_1_cnt,markers_238_1_cnt,markers_239_1_cnt,markers_240_1_cnt,markers_241_1_cnt,markers_242_1_cnt,markers_243_1_cnt,markers_244_1_cnt,markers_246_1_cnt,markers_247_1_cnt,markers_248_1_cnt,markers_249_1_cnt,markers_251_1_cnt,markers_252_1_cnt,markers_253_1_cnt,markers_254_1_cnt,markers_255_1_cnt,markers_256_1_cnt,markers_257_1_cnt,markers_258_1_cnt,markers_259_1_cnt,markers_260_1_cnt,markers_261_1_cnt,markers_262_1_cnt,markers_263_1_cnt,markers_264_1_cnt,markers_265_1_cnt,markers_266_1_cnt,markers_267_1_cnt,markers_268_1_cnt,markers_269_1_cnt,markers_270_1_cnt,markers_271_1_cnt,markers_272_1_cnt,markers_273_1_cnt,markers_274_1_cnt,markers_275_1_cnt,markers_276_1_cnt,markers_277_1_cnt,markers_279_1_cnt,markers_280_1_cnt,markers_281_1_cnt,markers_282_1_cnt,markers_283_1_cnt,markers_284_1_cnt,markers_285_1_cnt,markers_286_1_cnt,markers_287_1_cnt,markers_288_1_cnt,markers_289_1_cnt,markers_290_1_cnt,markers_291_1_cnt,markers_292_1_cnt,markers_293_1_cnt,markers_294_1_cnt,markers_295_1_cnt,markers_296_1_cnt,markers_297_1_cnt,markers_298_1_cnt,markers_299_1_cnt,markers_300_1_cnt,markers_301_1_cnt,markers_302_1_cnt,markers_303_1_cnt,markers_304_1_cnt,markers_305_1_cnt,markers_306_1_cnt,markers_307_1_cnt,markers_308_1_cnt,markers_309_1_cnt,markers_310_1_cnt,markers_312_1_cnt,markers_313_1_cnt,markers_314_1_cnt,markers_315_1_cnt,markers_316_1_cnt,markers_317_1_cnt,markers_318_1_cnt,markers_319_1_cnt,markers_320_1_cnt,markers_321_1_cnt,markers_322_1_cnt,markers_323_1_cnt,markers_324_1_cnt,markers_325_1_cnt,markers_326_1_cnt,markers_327_1_cnt,markers_328_1_cnt,markers_329_1_cnt,markers_330_1_cnt,markers_331_1_cnt,markers_332_1_cnt,markers_333_1_cnt,markers_334_1_cnt,markers_335_1_cnt,markers_336_1_cnt,markers_337_1_cnt,markers_338_1_cnt,markers_339_1_cnt,markers_340_1_cnt,markers_341_1_cnt,markers_342_1_cnt,markers_343_1_cnt,markers_344_1_cnt,markers_345_1_cnt,markers_346_1_cnt,markers_347_1_cnt,markers_348_1_cnt,markers_349_1_cnt,markers_350_1_cnt,markers_351_1_cnt,markers_352_1_cnt,markers_353_1_cnt,markers_354_1_cnt,markers_355_1_cnt,markers_356_1_cnt,markers_357_1_cnt,markers_358_1_cnt,markers_359_1_cnt,markers_360_1_cnt,markers_361_1_cnt,markers_362_1_cnt,markers_363_1_cnt,markers_364_1_cnt,markers_365_1_cnt,markers_366_1_cnt,markers_367_1_cnt,markers_368_1_cnt,markers_369_1_cnt,markers_370_1_cnt,markers_371_1_cnt,markers_372_1_cnt,markers_373_1_cnt,markers_374_1_cnt,markers_375_1_cnt,markers_376_1_cnt,markers_377_1_cnt,markers_378_1_cnt,markers_379_1_cnt,markers_380_1_cnt,markers_381_1_cnt,markers_382_1_cnt,markers_383_1_cnt,markers_384_1_cnt,markers_385_1_cnt,markers_386_1_cnt,markers_387_1_cnt,markers_388_1_cnt,markers_389_1_cnt,markers_390_1_cnt,markers_391_1_cnt,markers_392_1_cnt,markers_393_1_cnt,markers_394_1_cnt,markers_395_1_cnt,markers_396_1_cnt,markers_397_1_cnt,markers_398_1_cnt,markers_399_1_cnt,markers_400_1_cnt,markers_401_1_cnt,markers_402_1_cnt,markers_403_1_cnt,markers_404_1_cnt,markers_405_1_cnt,markers_406_1_cnt,markers_407_1_cnt,markers_408_1_cnt,markers_409_1_cnt,markers_410_1_cnt,markers_411_1_cnt,markers_412_1_cnt,target
0,-0.031107,-0.104697,-0.025089,-0.181242,-0.068119,-0.083954,-0.226446,-0.069071,-0.073598,-0.077716,-0.039401,-0.024738,-0.044995,-0.078226,-0.052385,NaN,-0.339619,-0.036073,-0.163505,-0.023135,-0.196457,-0.042695,-0.108565,-0.889347,NaN,-0.113154,NaN,-0.171718,-0.18704,-0.702268,-0.04025,-0.025074,NaN,-0.762698,-0.010933,-0.017246,-0.062057,-0.079599,-0.027791,-0.177535,-0.066134,-0.029

In [11]:
# отбрасываем константные значения
dataset_200_400 = variance_filter(dataset_200_400, 0.5)


In [12]:
# считаем корреляцию с таргетом
corr_200_400 = check_corr(dataset_200_400)
corr_200_400


markers_346_1_cnt    0.009750
markers_349_1_cnt    0.008966
markers_324_1_cnt    0.008129
markers_237_1_cnt    0.006892
markers_318_1_cnt    0.006859
                       ...   
markers_334_1_cnt   -0.005656
markers_376_1_cnt   -0.005760
markers_387_1_cnt   -0.006248
markers_249_1_cnt   -0.006698
markers_306_1_cnt   -0.006916
Length: 194, dtype: float64

### Столбцы с 400 по 599

In [13]:
# считываем необходимое количество столбцов
dataset_400_600 = read_data(dataset_train_path, 400, 600)
dataset_400_600.head()

,markers_413_1_cnt,markers_414_1_cnt,markers_415_1_cnt,markers_416_1_cnt,markers_417_1_cnt,markers_418_1_cnt,markers_419_1_cnt,markers_420_1_cnt,markers_421_1_cnt,markers_422_1_cnt,markers_423_1_cnt,markers_424_1_cnt,markers_425_1_cnt,markers_426_1_cnt,markers_427_1_cnt,markers_428_1_cnt,markers_429_1_cnt,markers_430_1_cnt,markers_431_1_cnt,markers_432_1_cnt,markers_433_1_cnt,markers_434_1_cnt,markers_435_1_cnt,markers_436_1_cnt,markers_437_1_cnt,markers_438_1_cnt,markers_439_1_cnt,markers_440_1_cnt,markers_441_1_cnt,markers_442_1_cnt,markers_443_1_cnt,markers_444_1_cnt,markers_445_1_cnt,markers_446_1_cnt,markers_447_1_cnt,markers_449_1_cnt,markers_450_1_cnt,markers_451_1_cnt,markers_452_1_cnt,markers_453_1_cnt,markers_454_1_cnt,markers_455_1_cnt,markers_456_1_cnt,markers_457_1_cnt,markers_458_1_cnt,markers_459_1_cnt,markers_460_1_cnt,markers_461_1_cnt,markers_462_1_cnt,markers_463_1_cnt,markers_465_1_cnt,markers_466_1_cnt,markers_467_1_cnt,markers_468_1_cnt,markers_469_1_cnt,markers_470_1_cnt,markers_471_1_cnt,markers_472_1_cnt,markers_473_1_cnt,markers_474_1_cnt,markers_475_1_cnt,markers_476_1_cnt,markers_477_1_cnt,markers_478_1_cnt,markers_479_1_cnt,markers_480_1_cnt,markers_481_1_cnt,markers_482_1_cnt,markers_483_1_cnt,markers_484_1_cnt,markers_485_1_cnt,markers_486_1_cnt,markers_487_1_cnt,markers_488_1_cnt,markers_489_1_cnt,markers_490_1_cnt,markers_491_1_cnt,markers_493_1_cnt,markers_494_1_cnt,markers_495_1_cnt,markers_496_1_cnt,markers_497_1_cnt,markers_498_1_cnt,markers_499_1_cnt,markers_500_1_cnt,markers_501_1_cnt,markers_502_1_cnt,markers_503_1_cnt,markers_504_1_cnt,markers_505_1_cnt,markers_506_1_cnt,markers_507_1_cnt,markers_508_1_cnt,markers_509_1_cnt,markers_510_1_cnt,markers_511_1_cnt,markers_512_1_cnt,markers_513_1_cnt,markers_514_1_cnt,markers_515_1_cnt,markers_516_1_cnt,markers_517_1_cnt,markers_518_1_cnt,markers_519_1_cnt,markers_520_1_cnt,markers_521_1_cnt,markers_522_1_cnt,markers_523_1_cnt,markers_524_1_cnt,markers_525_1_cnt,markers_526_1_cnt,markers_527_1_cnt,markers_528_1_cnt,markers_529_1_cnt,markers_530_1_cnt,markers_531_1_cnt,markers_532_1_cnt,markers_533_1_cnt,markers_534_1_cnt,markers_535_1_cnt,markers_536_1_cnt,markers_537_1_cnt,markers_538_1_cnt,markers_539_1_cnt,markers_540_1_cnt,markers_541_1_cnt,markers_542_1_cnt,markers_543_1_cnt,markers_544_1_cnt,markers_545_1_cnt,markers_546_1_cnt,markers_547_1_cnt,markers_548_1_cnt,markers_549_1_cnt,markers_550_1_cnt,markers_551_1_cnt,markers_552_1_cnt,markers_553_1_cnt,markers_554_1_cnt,markers_555_1_cnt,markers_556_1_cnt,markers_557_1_cnt,markers_558_1_cnt,markers_559_1_cnt,markers_560_1_cnt,markers_561_1_cnt,markers_562_1_cnt,markers_563_1_cnt,markers_564_1_cnt,markers_565_1_cnt,markers_566_1_cnt,markers_567_1_cnt,markers_568_1_cnt,markers_569_1_cnt,markers_570_1_cnt,markers_571_1_cnt,markers_572_1_cnt,markers_573_1_cnt,markers_574_1_cnt,markers_575_1_cnt,markers_576_1_cnt,markers_577_1_cnt,markers_578_1_cnt,markers_579_1_cnt,markers_580_1_cnt,markers_581_1_cnt,markers_582_1_cnt,markers_583_1_cnt,markers_584_1_cnt,markers_585_1_cnt,markers_586_1_cnt,markers_587_1_cnt,markers_588_1_cnt,markers_589_1_cnt,markers_590_1_cnt,markers_591_1_cnt,markers_592_1_cnt,markers_593_1_cnt,markers_594_1_cnt,markers_595_1_cnt,markers_596_1_cnt,markers_597_1_cnt,markers_598_1_cnt,markers_599_1_cnt,markers_600_1_cnt,markers_601_1_cnt,markers_602_1_cnt,markers_603_1_cnt,markers_604_1_cnt,markers_605_1_cnt,markers_606_1_cnt,markers_607_1_cnt,markers_608_1_cnt,markers_609_1_cnt,markers_610_1_cnt,markers_611_1_cnt,markers_612_1_cnt,markers_613_1_cnt,markers_614_1_cnt,markers_615_1_cnt,target
0,-0.094348,-0.114231,-0.103785,-0.014724,-0.106822,-0.016147,-0.204772,-0.018454,-0.12654,-0.087154,-0.08556,-0.668222,-0.126213,-0.257760,-0.011725,-0.160691,-0.023111,-0.109159,-0.333867,-0.028059,-0.049576,-0.402016,-0.043134,-0.135613,-0.037106,-0.628057,-0.139858,-0.108334,-0.081015,-0.346151,-0.139633,-0.332625,-0.170424,-0.104848,-0.059663,-0.016757,-0.037248,-0.105500,-0.139314,-0

In [14]:
# отбрасываем константные значения
dataset_400_600 = variance_filter(dataset_400_600, 0.5)


In [15]:
# считаем корреляцию с таргетом
corr_400_600 = check_corr(dataset_400_600)
corr_400_600

markers_476_1_cnt    0.012434
markers_434_1_cnt    0.011912
markers_567_1_cnt    0.011398
markers_506_1_cnt    0.011047
markers_508_1_cnt    0.010404
                       ...   
markers_465_1_cnt   -0.004971
markers_530_1_cnt   -0.005058
markers_611_1_cnt   -0.005200
markers_446_1_cnt   -0.005285
markers_444_1_cnt   -0.006422
Length: 195, dtype: float64

### Столбцы с 600 по 799

In [16]:
# считываем необходимое количество столбцов
dataset_600_800 = read_data(dataset_train_path, 600, 800)
dataset_600_800.head()

,markers_616_1_cnt,markers_617_1_cnt,markers_618_1_cnt,markers_619_1_cnt,markers_620_1_cnt,markers_621_1_cnt,markers_622_1_cnt,markers_623_1_cnt,markers_624_1_cnt,markers_626_1_cnt,markers_628_1_cnt,markers_629_1_cnt,markers_630_1_cnt,markers_631_1_cnt,markers_632_1_cnt,markers_633_1_cnt,markers_634_1_cnt,markers_635_1_cnt,markers_636_1_cnt,markers_637_1_cnt,markers_638_1_cnt,markers_639_1_cnt,markers_640_1_cnt,markers_641_1_cnt,markers_642_1_cnt,markers_643_1_cnt,markers_644_1_cnt,markers_645_1_cnt,markers_646_1_cnt,markers_647_1_cnt,markers_648_1_cnt,markers_649_1_cnt,markers_650_1_cnt,markers_651_1_cnt,markers_652_1_cnt,markers_653_1_cnt,markers_654_1_cnt,markers_655_1_cnt,markers_656_1_cnt,markers_657_1_cnt,markers_658_1_cnt,markers_659_1_cnt,markers_660_1_cnt,markers_661_1_cnt,markers_662_1_cnt,markers_663_1_cnt,markers_664_1_cnt,markers_665_1_cnt,markers_666_1_cnt,markers_667_1_cnt,markers_668_1_cnt,markers_669_1_cnt,markers_670_1_cnt,markers_671_1_cnt,markers_672_1_cnt,markers_673_1_cnt,markers_674_1_cnt,markers_675_1_cnt,markers_677_1_cnt,markers_678_1_cnt,markers_679_1_cnt,markers_680_1_cnt,markers_681_1_cnt,markers_682_1_cnt,markers_683_1_cnt,markers_684_1_cnt,markers_686_1_cnt,markers_687_1_cnt,markers_688_1_cnt,markers_689_1_cnt,markers_690_1_cnt,markers_691_1_cnt,markers_692_1_cnt,markers_693_1_cnt,markers_696_1_cnt,markers_697_1_cnt,markers_698_1_cnt,markers_699_1_cnt,markers_700_1_cnt,markers_701_1_cnt,markers_702_1_cnt,markers_703_1_cnt,markers_704_1_cnt,markers_705_1_cnt,markers_706_1_cnt,markers_707_1_cnt,markers_708_1_cnt,markers_709_1_cnt,markers_710_1_cnt,markers_711_1_cnt,markers_712_1_cnt,markers_713_1_cnt,markers_714_1_cnt,markers_715_1_cnt,markers_716_1_cnt,markers_717_1_cnt,markers_718_1_cnt,markers_719_1_cnt,markers_720_1_cnt,markers_721_1_cnt,markers_722_1_cnt,markers_723_1_cnt,markers_724_1_cnt,markers_725_1_cnt,markers_726_1_cnt,markers_727_1_cnt,markers_728_1_cnt,markers_729_1_cnt,markers_730_1_cnt,markers_731_1_cnt,markers_732_1_cnt,markers_733_1_cnt,markers_734_1_cnt,markers_735_1_cnt,markers_736_1_cnt,markers_737_1_cnt,markers_738_1_cnt,markers_739_1_cnt,markers_740_1_cnt,markers_741_1_cnt,markers_742_1_cnt,markers_743_1_cnt,markers_744_1_cnt,markers_745_1_cnt,markers_746_1_cnt,markers_747_1_cnt,markers_748_1_cnt,markers_749_1_cnt,markers_750_1_cnt,markers_751_1_cnt,markers_752_1_cnt,markers_753_1_cnt,markers_754_1_cnt,markers_755_1_cnt,markers_756_1_cnt,markers_757_1_cnt,markers_758_1_cnt,markers_759_1_cnt,markers_760_1_cnt,markers_761_1_cnt,markers_762_1_cnt,markers_763_1_cnt,markers_764_1_cnt,markers_765_1_cnt,markers_766_1_cnt,markers_767_1_cnt,markers_768_1_cnt,markers_769_1_cnt,markers_770_1_cnt,markers_771_1_cnt,markers_772_1_cnt,markers_773_1_cnt,markers_774_1_cnt,markers_775_1_cnt,markers_776_1_cnt,markers_777_1_cnt,markers_778_1_cnt,markers_779_1_cnt,markers_780_1_cnt,markers_781_1_cnt,markers_782_1_cnt,markers_783_1_cnt,markers_784_1_cnt,markers_785_1_cnt,markers_786_1_cnt,markers_787_1_cnt,markers_788_1_cnt,markers_789_1_cnt,markers_790_1_cnt,markers_791_1_cnt,markers_792_1_cnt,markers_793_1_cnt,markers_794_1_cnt,markers_795_1_cnt,markers_796_1_cnt,markers_797_1_cnt,markers_798_1_cnt,markers_799_1_cnt,markers_800_1_cnt,markers_801_1_cnt,markers_802_1_cnt,markers_803_1_cnt,markers_804_1_cnt,markers_805_1_cnt,spas_symptoms_agr_0_1_std,spas_symptoms_agr_1_1_sum,spas_symptoms_agr_2_3_avg,spas_symptoms_agr_3_3_std,spas_symptoms_agr_4_3_sum,spas_symptoms_agr_5_6_avg,spas_symptoms_agr_6_6_std,spas_symptoms_agr_7_6_sum,spas_symptoms_agr_8_12_avg,spas_symptoms_agr_9_12_std,spas_symptoms_agr_10_12_sum,spas_symptoms_agr_11_1_avg,spas_symptoms_agr_12_1_std,spas_symptoms_agr_13_1_sum,spas_symptoms_agr_14_3_avg,spas_symptoms_agr_15_3_std,target
0,-0.179294,-0.065754,-0.238323,-0.104036,-0.078483,-0.116726,-0.178096,-0.215031,NaN,-0.092402,NaN,-0.15854,-0.111964,-0.024089,-0.019059,-0.015788,-0.045439,-0.011497,NaN,-0.054198,-0.087955,-0.12999,-0.110440,-0.055536,-0.03872,-0.086761,-0.051684,-0.0

In [17]:
# отбрасываем константные значения
dataset_600_800 = variance_filter(dataset_600_800, 0.5)


In [18]:
# считаем корреляцию с таргетом
corr_600_800 = check_corr(dataset_600_800)
corr_600_800

spas_symptoms_agr_10_12_sum    0.022697
spas_symptoms_agr_14_3_avg     0.022478
spas_symptoms_agr_13_1_sum     0.022373
spas_symptoms_agr_11_1_avg     0.022152
spas_symptoms_agr_8_12_avg     0.021893
                                 ...   
spas_symptoms_agr_7_6_sum     -0.020821
spas_symptoms_agr_4_3_sum     -0.021741
spas_symptoms_agr_1_1_sum     -0.022496
spas_symptoms_agr_2_3_avg     -0.023163
spas_symptoms_agr_5_6_avg     -0.023248
Length: 198, dtype: float64

### Столбцы с 800 по 930

In [19]:
# считываем необходимое количество столбцов
dataset_800_930 = read_data(dataset_train_path, 800, 931)
dataset_800_930.head()

,spas_symptoms_agr_16_3_sum,spas_symptoms_agr_17_6_avg,spas_symptoms_agr_18_6_std,spas_symptoms_agr_19_6_sum,materials_details_0_1_ctg,communication_availability_0_1_flg,payments_details_0_1_cnt,payments_details_1_3_cnt,payments_details_2_6_cnt,payments_details_3_1_cnt,payments_details_4_3_cnt,payments_details_5_6_cnt,payments_details_6_1_cnt,payments_details_7_3_cnt,payments_details_8_6_cnt,payments_details_9_1_cnt,payments_details_10_3_cnt,payments_details_11_6_cnt,payments_details_12_1_cnt,payments_details_13_3_cnt,payments_details_14_6_cnt,payments_details_15_1_cnt,payments_details_16_1d3_avg,payments_details_17_1d3_cnt,payments_details_18_1d3_sum,payments_details_19_1d6_avg,payments_details_20_1d6_cnt,payments_details_21_1d6_sum,payments_details_22_3_cnt,payments_details_23_3d6_avg,payments_details_24_3d6_cnt,payments_details_25_3d6_sum,payments_details_26_6_cnt,payments_details_27_1_sumpct,payments_details_28_3_sumpct,payments_details_29_6_sumpct,user_devices_0_1_cnt,communication_availability_1_1_ctg,user_devices_1_1_cnt,communication_availability_2_1_flg,materials_details_1_1_ctg,user_lifetime_0_1_ctg,user_lifetime_1_1_flg,materials_details_2_1_cnt,communication_availability_3_1_flg,materials_details_3_1_dt,materials_details_4_1_dt,materials_details_5_1_flg,materials_details_6_1_num,materials_details_7_1_flg,materials_details_8_1_flg,user_devices_2_1_cnt,arpu_0_1_sum,arpu_1_3_avg,arpu_2_6_avg,balance_details_0_1_num,charges_details_0_1_sum,charges_details_1_3_avg,charges_details_2_6_avg,charges_details_3_1_sum,charges_details_4_3_avg,charges_details_5_6_avg,charges_details_6_1_sum,charges_details_7_3_avg,charges_details_8_6_avg,charges_details_9_1_sum,charges_details_10_3_avg,charges_details_11_6_avg,tariff_plans_0_1_num,tariff_plans_1_1_num,charges_details_12_1_sum,charges_details_13_3_avg,charges_details_14_6_avg,tariff_plans_2_1_num,tariff_plans_3_1_num,charges_details_15_1_sum,charges_details_16_3_avg,charges_details_17_6_avg,tariff_plans_4_1_num,tariff_plans_5_1_num,charges_details_18_1_sum,charges_details_19_3_avg,charges_details_20_6_avg,tariff_plans_6_1_num,tariff_plans_7_1_num,charges_details_21_1_sum,charges_details_22_3_avg,charges_details_23_6_avg,charges_details_24_1_sum,charges_details_25_3_avg,charges_details_26_6_avg,charges_details_27_1_sum,charges_details_28_3_avg,charges_details_29_6_avg,payments_details_30_1_sum,payments_details_31_3_sum,payments_details_32_6_sum,payments_details_33_1_sum,payments_details_34_3_sum,payments_details_35_6_sum,payments_details_36_1_sum,payments_details_37_3_sum,payments_details_38_6_sum,payments_details_39_1_sum,payments_details_40_3_sum,payments_details_41_6_sum,payments_details_42_1_sum,payments_details_43_3_sum,payments_details_44_6_sum,payments_details_45_1_avg,payments_details_46_1_sum,payments_details_47_3_avg,payments_details_48_3_sum,payments_details_49_6_avg,payments_details_50_6_sum,plan_costs_0_1_sum,plan_costs_1_3_avg,plan_costs_2_6_avg,plan_costs_3_1_sum,plan_costs_4_3_avg,plan_costs_5_6_avg,plan_costs_6_1_sum,plan_costs_7_3_avg,plan_costs_8_6_avg,charges_details_30_1_sum,charges_details_31_3_avg,charges_details_32_6_avg,tariff_plans_8_1_num,tariff_plans_9_1_num,vas_details_0_1_sum,vas_details_1_3_sum,target
0,-0.413926,-0.415941,-0.56747,-0.410849,1,1,-0.484809,-0.582865,-0.621387,0.277911,0.359126,0.404608,0.0,0.0,0.0,-0.00409,-0.007544,-0.009613,0.0,0.0,0.0,-0.004218,0.358846,0.009302,0.043081,0.388700,-0.080449,-0.015497,0.015892,0.172597,-0.107577,-0.022152,0.044289,-0.316965,-0.623833,-0.722535,-0.310186,5,-0.03573,1,1,0,-1,NaN,1,NaN,NaN,-1,NaN,-1,-1,-0.039662,-1.340015,-1.388009,-1.418177,-0.067160,-1.482161,-1.511285,-1.537175,-1.483398,-1.512404,-1.538253,0.125476,0.104903,0.112379,0.125454,0.104868,0.112337,NaN,NaN,-0.850738,-0.8548,-0.857056,NaN,NaN,-1.929450,-1.986848,-2.050207,-2.232055,-2.065076,1.375516,1.396115,1.426084,-0.301287,-1.479226,0.083358,0.059508,0.082743,0.083324,0.059462,0.082690,0.0,0.0,0.0,-0.396153,-0.533037,-0.593226,-0.3595

In [20]:
# отбрасываем константные значения
dataset_800_930 = variance_filter(dataset_800_930, 0.5)


In [21]:
# считаем корреляцию с таргетом
corr_800_930 = check_corr(dataset_800_930)
corr_800_930

materials_details_4_1_dt     0.063585
materials_details_3_1_dt     0.059938
materials_details_6_1_num    0.059930
charges_details_13_3_avg     0.031386
charges_details_14_6_avg     0.031338
                               ...   
charges_details_10_3_avg    -0.023347
charges_details_7_3_avg     -0.023352
charges_details_11_6_avg    -0.023568
charges_details_8_6_avg     -0.023573
materials_details_2_1_cnt   -0.041085
Length: 110, dtype: float64